In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem

import warnings
warnings.simplefilter('ignore')

from flask import Flask
import json

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Input, Dot
from keras.layers.merge import Concatenate
from keras import regularizers

from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier

from additions.sber import *
from additions.attention_layer import *

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('data/app_review_rating_train.csv')

train_data.drop(['Date', 'AppName', 'Language'], axis = 1, inplace = True)
train_data['Review'] = train_data['Review'].map(preprocess)
train_data['Title'] = train_data['Title'].map(preprocess)

val_data = pd.DataFrame()
for i in range(1,6):
    val_data = val_data.append(train_data[train_data['Rating'] == i].sample(int(train_data['Rating'].value_counts().loc[i] * 0.1),random_state = 7))
train_data.drop(val_data.index,inplace=True)

# m = Mystem()
# train_data['Review'] = train_data['Review'].map(lambda x: ''.join(m.lemmatize(str(x))[:-1]))
# train_data['Title'] = train_data['Title'].map(lambda x: ''.join(m.lemmatize(str(x))[:-1]))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train_data['Review']) + list(train_data['Title']) + list(val_data['Title']) + list(val_data['Review']))
print('Words count -', len(tokenizer.word_index))
print('98% percentile - ' + str(np.percentile(train_data['Review'].map(lambda x: len(x.split(' '))), 98)))

Words count - 38161
98% percentile - 47.0


In [3]:
max_features = 40000
maxlen = 50
embedding_dim = 50

label_binarizer = LabelBinarizer()
label_binarizer.fit([1,2,3,4,5])
y_train = label_binarizer.transform(train_data['Rating'])
y_val = label_binarizer.transform(val_data['Rating'])

In [4]:
X_train = list()
X_val = list()

for i in range(0,len(train_data)):
    X_train.append(str(train_data['Title'].iloc[i]) + ' ENDTITLE ' + str(train_data['Review'].iloc[i]))
for i in range(0,len(val_data)):
    X_val.append(str(val_data['Title'].iloc[i]) + ' ENDTITLE ' + str(val_data['Review'].iloc[i]))
    
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(list(X_train), maxlen)

X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(list(X_val), maxlen)

In [5]:
input_layer = Input(shape=(maxlen,))
embeddings = Embedding(max_features, embedding_dim, input_length=maxlen, embeddings_regularizer = regularizers.l1(10e-5))(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embeddings)

attention = AttentionWithContext()(lstm_layer)

model = Dot(axes=0)([lstm_layer,attention])
model = Dropout(0.5)(model)

out = Dense(5, activation='softmax')(model)

model = Model(inputs=input_layer, outputs=out)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics = ['accuracy'])

model.fit(
    X_train, y_train, 
    batch_size = 1024, 
    epochs = 20,
    validation_data = (X_val,y_val)
)

Train on 68655 samples, validate on 7626 samples
Epoch 1/20
68655/68655 [==============================] - 6s - loss: 2.5094 - acc: 0.6668 - val_loss: 1.0191 - val_acc: 0.6783
Epoch 2/20
68655/68655 [==============================] - 5s - loss: 0.8624 - acc: 0.7139 - val_loss: 0.7473 - val_acc: 0.7568
Epoch 3/20
68655/68655 [==============================] - 5s - loss: 0.7505 - acc: 0.7640 - val_loss: 0.7424 - val_acc: 0.7689
Epoch 4/20
68655/68655 [==============================] - 5s - loss: 0.7219 - acc: 0.7757 - val_loss: 0.7090 - val_acc: 0.7813
Epoch 5/20
68655/68655 [==============================] - 5s - loss: 0.7019 - acc: 0.7824 - val_loss: 0.6802 - val_acc: 0.7868
Epoch 6/20
68655/68655 [==============================] - 5s - loss: 0.6794 - acc: 0.7884 - val_loss: 0.6682 - val_acc: 0.7922
Epoch 7/20
68655/68655 [==============================] - 5s - loss: 0.6691 - acc: 0.7915 - val_loss: 0.6679 - val_acc: 0.7905
Epoch 8/20
68655/68655 [==============================] - 5s -

In [6]:
# together
X_train = list()
X_val = list()
for i in range(0,len(train_data)):
    X_train.append(str(train_data['Title'].iloc[i]) + ' ' + str(train_data['Review'].iloc[i]))
    
for i in range(0,len(val_data)):
    X_val.append(str(val_data['Title'].iloc[i]) + ' ' + str(val_data['Review'].iloc[i]))
    
vectorizer = CountVectorizer(ngram_range=(1,2))

X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)

y_train = list(train_data['Rating'])
y_val = list(val_data['Rating'])

In [7]:
ridge = RidgeClassifier(alpha = 0.01)
ridge.fit(X_train,y_train)

RidgeClassifier(alpha=0.01, class_weight=None, copy_X=True,
        fit_intercept=True, max_iter=None, normalize=False,
        random_state=None, solver='auto', tol=0.001)

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 50)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 50, 50)        2000000     input_1[0][0]                    
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 50, 64)        29440       embedding_1[0][0]                
____________________________________________________________________________________________________
attention_with_context_1 (Attent (None, 64)            4224        lstm_1[0][0]                     
___________________________________________________________________________________________

In [ ]:
dictionary = dict(map(reversed, tokenizer.word_index.items()))
dictionary[0] = 'unknown'

In [ ]:
app = Flask("demo")
@app.route('/')
def index():
    return ("Prediction model")

@app.route('/predict/<text>')
def predict(text):
    print(text)
    js = {}
    
    text = preprocess(text)
    
    vec = tokenizer.texts_to_sequences([text])
    
    pad = pad_sequences(list(vec), maxlen)
    pred = model.predict([pad])[0]
    
    prob = np.max(pred)
    
    pred = np.argmax(pred)
    pred = pred + 1
    
    activations = get_activations(model,pad,layer_name='attention_with_context_1')
    
    for i,value in enumerate(pad[0]):
        if value != 0:
            new_i = i
            new = pad[0][i:]
            activations = activations[0][0][new_i:]
            break
    
    if pad[0][-1] == 0:
        js['prob'] = 0
        js['class'] = 0
        js['words'] = [{
            "value": 'Unknown',
            "color": '#8EFF8E'
        }]
    else:
        sentence = [dictionary[word] for word in new]
        jswords = []
            
        if pred == 5: 
            for i, word in enumerate(sentence):
                jswords.append({
                    "value": word,
                    "color": '#FFFFFF'
                })
                jswords.append({
                    "value": ' ',
                    "color": '#FFFFFF'
                })
        else:
            
            for i, word in enumerate(sentence):
                coef = activations[i] * 255 // 1
                if coef  > 235:
                    coef = str('#FF' + '0' + hex(255 - int(coef))[2:].upper() + '0' + hex(255 - int(coef))[2:].upper()) 
                else:
                    coef = str('#FF' + hex(255 - int(coef))[2:].upper() + hex(255 - int(coef))[2:].upper())
                jswords.append({
                    "value": word,
                    "color": coef
                })
                jswords.append({
                    "value": ' ',
                    "color": '#FFFFFF'
                })

        js['prob'] = float(prob)
        js['class'] = int(pred)
        js['words'] = jswords
    
    return(json.dumps(js,ensure_ascii=False))


@app.route('/predict2/<text>')
def predict2(text):
    js = {}
    
    text = preprocess(text)
    
    vec = vectorizer.transform([text])
    pred = ridge.predict(vec)[0]
    
    js['class'] = int(pred)
    js['words'] = text
    
    return(json.dumps(js,ensure_ascii=False))

app.run(host="0.0.0.0", port="9000")

 * Running on http://0.0.0.0:9000/ (Press CTRL+C to quit)


Сбербанк Отлично работает, очень нравит


192.168.88.112 - - [13/Dec/2017 17:40:44] "GET /predict/%D0%A1%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA%20%D0%9E%D1%82%D0%BB%D0%B8%D1%87%D0%BD%D0%BE%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%B5%D1%82,%20%D0%BE%D1%87%D0%B5%D0%BD%D1%8C%20%D0%BD%D1%80%D0%B0%D0%B2%D0%B8%D1%82 HTTP/1.1" 200 -
192.168.88.112 - - [13/Dec/2017 17:40:44] "GET /predict2/%D0%A1%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA%20%D0%9E%D1%82%D0%BB%D0%B8%D1%87%D0%BD%D0%BE%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%B5%D1%82,%20%D0%BE%D1%87%D0%B5%D0%BD%D1%8C%20%D0%BD%D1%80%D0%B0%D0%B2%D0%B8%D1%82 HTTP/1.1" 200 -
192.168.88.112 - - [13/Dec/2017 17:40:45] "GET /predict/%D0%A1%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA%20%D0%9E%D1%82%D0%BB%D0%B8%D1%87%D0%BD%D0%BE%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%B5%D1%82,%20%D0%BE%D1%87%D0%B5%D0%BD%D1%8C%20%D0%BD%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D1%81%D1%8F! HTTP/1.1" 200 -
192.168.88.112 - - [13/Dec/2017 17:40:45] "GET /predict2/%D0%A1%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA%20%D0%9E%D

Сбербанк Отлично работает, очень нравится!


192.168.88.112 - - [13/Dec/2017 17:41:46] "GET /predict/%D0%A1%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA%20Ghb HTTP/1.1" 200 -
192.168.88.112 - - [13/Dec/2017 17:41:46] "GET /predict2/%D0%A1%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA%20Ghb HTTP/1.1" 200 -


...

Ужасно Исправьт


192.168.88.112 - - [13/Dec/2017 17:48:41] "GET /predict/%D0%A3%D0%B6%D0%B0%D1%81%D0%BD%D0%BE%20%D0%9C%D0%BD%D0%B5%20%D0%BD%D0%B5%20%D0%BD%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D1%81%D1%8F HTTP/1.1" 200 -
192.168.88.112 - - [13/Dec/2017 17:48:41] "GET /predict2/%D0%A3%D0%B6%D0%B0%D1%81%D0%BD%D0%BE%20%D0%9C%D0%BD%D0%B5%20%D0%BD%D0%B5%20%D0%BD%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D1%81%D1%8F HTTP/1.1" 200 -


Ужасно Мне не нравится


192.168.88.112 - - [13/Dec/2017 17:48:41] "GET /predict/%D0%A3%D0%B6%D0%B0%D1%81%D0%BD%D0%BE%20%D0%9C%D0%BD%D0%B5%20%D0%BD%D0%B5%20%D0%BD%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D1%81%D1%8F! HTTP/1.1" 200 -
192.168.88.112 - - [13/Dec/2017 17:48:41] "GET /predict2/%D0%A3%D0%B6%D0%B0%D1%81%D0%BD%D0%BE%20%D0%9C%D0%BD%D0%B5%20%D0%BD%D0%B5%20%D0%BD%D1%80%D0%B0%D0%B2%D0%B8%D1%82%D1%81%D1%8F! HTTP/1.1" 200 -


Ужасно Мне не нравится!
